In [1]:
import spark_env

spark = spark_env.create_spark_session('pyspark intermediate')

In [2]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [40]:
df = spark.read.format('csv')\
                .option('inferSchema', True)\
                .option('header',True)\
                .load('Bigmart Sales.csv')

## Union and UnionByName

In [5]:
data1 = [
    ('1','sid'),
    ('2','kad')
]
schema = StructType([
    StructField('id',StringType(),True),
    StructField('name',StringType(),True),
])

data2 = [
    ('3', 'Tim'),
    ('4', 'Jim')
]

df1 = spark.createDataFrame(data1,schema)
df2 = spark.createDataFrame(data2,schema)

In [6]:
df1.show()

+---+----+
| id|name|
+---+----+
|  1| sid|
|  2| kad|
+---+----+



In [7]:
df2.show()

+---+----+
| id|name|
+---+----+
|  3| Tim|
|  4| Jim|
+---+----+



In [15]:
df_union = df1.union(df2)

In [13]:
data3 = [
    ('hara','6'),
    ('hari','7')
]

schema_ubn = StructType([
    StructField('name',StringType(),True),
    StructField('id',StringType(),True)
])

df3 = spark.createDataFrame(data3,schema_ubn)

In [14]:
df3.show()

+----+---+
|name| id|
+----+---+
|hara|  6|
|hari|  7|
+----+---+



This gives incorrect union without checking the column names. This is not acceptable. That is why we need to use unionByName()

In [17]:
df_union.union(df3).show()

+----+----+
|  id|name|
+----+----+
|   1| sid|
|   2| kad|
|   3| Tim|
|   4| Jim|
|hara|   6|
|hari|   7|
+----+----+



In [18]:
df_union.unionByName(df3).show()

+---+----+
| id|name|
+---+----+
|  1| sid|
|  2| kad|
|  3| Tim|
|  4| Jim|
|  6|hara|
|  7|hari|
+---+----+



## String Functions

#### initCap(), upper(), lower()

In [20]:
df.select(initcap(col('Item_Type'))).show()

+--------------------+
|  initcap(Item_Type)|
+--------------------+
|               Dairy|
|         Soft Drinks|
|                Meat|
|Fruits And Vegeta...|
|           Household|
|        Baking Goods|
|         Snack Foods|
|         Snack Foods|
|        Frozen Foods|
|        Frozen Foods|
|Fruits And Vegeta...|
|               Dairy|
|Fruits And Vegeta...|
|         Snack Foods|
|Fruits And Vegeta...|
|           Breakfast|
|  Health And Hygiene|
|           Breakfast|
|         Hard Drinks|
|               Dairy|
+--------------------+
only showing top 20 rows



In [21]:
df.select(upper(col('Item_Type'))).show()

+--------------------+
|    upper(Item_Type)|
+--------------------+
|               DAIRY|
|         SOFT DRINKS|
|                MEAT|
|FRUITS AND VEGETA...|
|           HOUSEHOLD|
|        BAKING GOODS|
|         SNACK FOODS|
|         SNACK FOODS|
|        FROZEN FOODS|
|        FROZEN FOODS|
|FRUITS AND VEGETA...|
|               DAIRY|
|FRUITS AND VEGETA...|
|         SNACK FOODS|
|FRUITS AND VEGETA...|
|           BREAKFAST|
|  HEALTH AND HYGIENE|
|           BREAKFAST|
|         HARD DRINKS|
|               DAIRY|
+--------------------+
only showing top 20 rows



In [24]:
df.select(lower(col('Item_Type')).alias('Lower Item_Type')).show()

+--------------------+
|     Lower Item_Type|
+--------------------+
|               dairy|
|         soft drinks|
|                meat|
|fruits and vegeta...|
|           household|
|        baking goods|
|         snack foods|
|         snack foods|
|        frozen foods|
|        frozen foods|
|fruits and vegeta...|
|               dairy|
|fruits and vegeta...|
|         snack foods|
|fruits and vegeta...|
|           breakfast|
|  health and hygiene|
|           breakfast|
|         hard drinks|
|               dairy|
+--------------------+
only showing top 20 rows



## Date Functions

1. current_date()
2. date_add()
3. date_sub()
4. date_diff()
5. date_format()

In [41]:
df = df.withColumn('current_date',current_date())
df.show(2)

+---------------+-----------+----------------+---------------+-----------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+------------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|  Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|current_date|
+---------------+-----------+----------------+---------------+-----------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+------------+
|          FDA15|        9.3|         Low Fat|    0.016047301|      Dairy|249.8092|           OUT049|                     1999|     Medium|              Tier 1|Supermarket Type1|         3735.138|  2025-07-12|
|          DRC01|       5.92|         Regular|    0.019278216|Soft Drinks| 48.2692|           OUT018|                     2009|     Medium|              Tier 3|

In [44]:
df = df.withColumn('week_later',date_add('current_date',7))
df.show(2)

+---------------+-----------+----------------+---------------+-----------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+------------+----------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|  Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|current_date|week_later|
+---------------+-----------+----------------+---------------+-----------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+------------+----------+
|          FDA15|        9.3|         Low Fat|    0.016047301|      Dairy|249.8092|           OUT049|                     1999|     Medium|              Tier 1|Supermarket Type1|         3735.138|  2025-07-12|2025-07-19|
|          DRC01|       5.92|         Regular|    0.019278216|Soft Drinks| 48.2692|           OUT018|               

In [48]:
df = df.withColumn('week_before',date_sub('current_date',7))
df.show(2)

+---------------+-----------+----------------+---------------+-----------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+------------+----------+-----------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|  Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|current_date|week_later|week_before|
+---------------+-----------+----------------+---------------+-----------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+------------+----------+-----------+
|          FDA15|        9.3|         Low Fat|    0.016047301|      Dairy|249.8092|           OUT049|                     1999|     Medium|              Tier 1|Supermarket Type1|         3735.138|  2025-07-12|2025-07-19| 2025-07-05|
|          DRC01|       5.92|         Regular|    0.019278216|Soft D

In [50]:
df = df.withColumn('difference_of_dates',date_diff(col('week_later'),col('week_before')))
df.show(2)

+---------------+-----------+----------------+---------------+-----------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+------------+----------+-----------+-------------------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|  Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|current_date|week_later|week_before|difference_of_dates|
+---------------+-----------+----------------+---------------+-----------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+------------+----------+-----------+-------------------+
|          FDA15|        9.3|         Low Fat|    0.016047301|      Dairy|249.8092|           OUT049|                     1999|     Medium|              Tier 1|Supermarket Type1|         3735.138|  2025-07-12|2025-07-19| 2025-07-05|         

In [52]:
df = df.withColumn('current_date',date_format('current_date','MM-dd-yyyy'))
df.show(2)

+---------------+-----------+----------------+---------------+-----------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+------------+----------+-----------+-------------------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|  Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|current_date|week_later|week_before|difference_of_dates|
+---------------+-----------+----------------+---------------+-----------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+------------+----------+-----------+-------------------+
|          FDA15|        9.3|         Low Fat|    0.016047301|      Dairy|249.8092|           OUT049|                     1999|     Medium|              Tier 1|Supermarket Type1|         3735.138|  07-12-2025|2025-07-19| 2025-07-05|         

## Handling Null Values

1. Dropping Nulls
- dropna('all') -> Drops rows with all the values as null
- dropna('any') -> Drops rows with any value in the row that is null
- dropna(subset=[]) -> Drops rows for the null values in the column mentioned in the subset
2. Filling Nulls
- fillna(value) -> Replaces all the null values in the date with the value given
- fillna(value, subset=[]) -> Replaces the values only in the specified column

In [53]:
df.dropna('all').count()

8523

In [54]:
df.dropna('any').count()

4650

In [55]:
df.dropna(subset=['Outlet_Size']).count()

6113

In [56]:
df.fillna('Not Available').show(10)

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-------------+--------------------+-----------------+-----------------+------------+----------+-----------+-------------------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|  Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|current_date|week_later|week_before|difference_of_dates|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-------------+--------------------+-----------------+-----------------+------------+----------+-----------+-------------------+
|          FDA15|        9.3|         Low Fat|    0.016047301|               Dairy|249.8092|           OUT049|                     1999|       Medium|              Tier 1|Supermarket Type1|         3735.138| 

In [59]:
df.fillna('Not Available', subset=['Outlet_Size']).show(5)

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-------------+--------------------+-----------------+-----------------+------------+----------+-----------+-------------------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|  Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|current_date|week_later|week_before|difference_of_dates|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-------------+--------------------+-----------------+-----------------+------------+----------+-----------+-------------------+
|          FDA15|        9.3|         Low Fat|    0.016047301|               Dairy|249.8092|           OUT049|                     1999|       Medium|              Tier 1|Supermarket Type1|         3735.138| 

## String Functions

### Split and Indexing

1. Split the values in Outlet_Type and then take the second value and replace the value for the Outlet_Type column.

In [62]:
df.withColumn('Outlet_Type',split('Outlet_Type',' ')[1]).show(5)

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------+-----------------+------------+----------+-----------+-------------------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|Outlet_Type|Item_Outlet_Sales|current_date|week_later|week_before|difference_of_dates|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------+-----------------+------------+----------+-----------+-------------------+
|          FDA15|        9.3|         Low Fat|    0.016047301|               Dairy|249.8092|           OUT049|                     1999|     Medium|              Tier 1|      Type1|         3735.138|  07-12-2025|2025-07-19| 2025-07-

### Explode
If the value in a column is in array, we can use explode() to create different rows for each value in the list

In [63]:
df.withColumn('Outlet_Type',split('Outlet_Type',' '))\
    .withColumn('Outlet_Type',explode('Outlet_Type')).show(5)

+---------------+-----------+----------------+---------------+-----------+--------+-----------------+-------------------------+-----------+--------------------+-----------+-----------------+------------+----------+-----------+-------------------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|  Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|Outlet_Type|Item_Outlet_Sales|current_date|week_later|week_before|difference_of_dates|
+---------------+-----------+----------------+---------------+-----------+--------+-----------------+-------------------------+-----------+--------------------+-----------+-----------------+------------+----------+-----------+-------------------+
|          FDA15|        9.3|         Low Fat|    0.016047301|      Dairy|249.8092|           OUT049|                     1999|     Medium|              Tier 1|Supermarket|         3735.138|  07-12-2025|2025-07-19| 2025-07-05|                 14|
|          F

### Array_Contains
Check if a value is present in an array in the column

In [64]:
df.withColumn('Outlet_Type',split('Outlet_Type',' '))\
    .withColumn('Type1_flag',array_contains('Outlet_Type','Type1')).show(5)

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+--------------------+-----------------+------------+----------+-----------+-------------------+----------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|         Outlet_Type|Item_Outlet_Sales|current_date|week_later|week_before|difference_of_dates|Type1_flag|
+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+--------------------+-----------------+------------+----------+-----------+-------------------+----------+
|          FDA15|        9.3|         Low Fat|    0.016047301|               Dairy|249.8092|           OUT049|                     1999|     Medium|              Tier 1|[Su

## Group_By

In [69]:
# Find the sum of the Item_MRP by each Item_Type
df.groupBy('Item_Type').agg(round(sum(col('Item_MRP')),2).alias('Total MRP')).show()

+--------------------+---------+
|           Item_Type|Total MRP|
+--------------------+---------+
|       Starchy Foods| 21880.03|
|        Baking Goods| 81894.74|
|              Breads| 35379.12|
|Fruits and Vegeta...|178124.08|
|                Meat| 59449.86|
|         Hard Drinks| 29334.68|
|         Soft Drinks| 58514.16|
|           Household|135976.53|
|           Breakfast|  15596.7|
|               Dairy|101276.46|
|         Snack Foods|175433.92|
|              Others| 22451.89|
|             Seafood|  9077.87|
|              Canned| 90706.73|
|        Frozen Foods|118558.88|
|  Health and Hygiene| 68025.84|
+--------------------+---------+



In [71]:
# Find the average of the Item_MRP by each Item_Type
df.groupBy('Item_Type').agg(round(avg(col('Item_MRP')),2).alias('Total MRP')).show()

+--------------------+---------+
|           Item_Type|Total MRP|
+--------------------+---------+
|       Starchy Foods|   147.84|
|        Baking Goods|   126.38|
|              Breads|   140.95|
|Fruits and Vegeta...|   144.58|
|                Meat|   139.88|
|         Hard Drinks|   137.08|
|         Soft Drinks|   131.49|
|           Household|   149.42|
|           Breakfast|   141.79|
|               Dairy|    148.5|
|         Snack Foods|   146.19|
|              Others|   132.85|
|             Seafood|   141.84|
|              Canned|   139.76|
|        Frozen Foods|    138.5|
|  Health and Hygiene|   130.82|
+--------------------+---------+



In [73]:
# Find the sum of the Item_MRP by each Item_Type and Outlet_Size
df.groupBy('Item_Type', 'Outlet_Size')\
    .agg(round(sum(col('Item_MRP')),2)\
    .alias('Total MRP'))\
    .dropna(subset=['Outlet_Size'])\
    .show()

+--------------------+-----------+---------+
|           Item_Type|Outlet_Size|Total MRP|
+--------------------+-----------+---------+
|       Starchy Foods|     Medium|  7124.14|
|Fruits and Vegeta...|     Medium| 59047.22|
|        Frozen Foods|       High| 12588.29|
|         Soft Drinks|       High|  6456.17|
|           Breakfast|      Small|  3917.04|
|                Meat|     Medium| 20326.45|
|Fruits and Vegeta...|       High| 20671.35|
|                Meat|       High|  5627.04|
|        Baking Goods|       High|  9431.75|
|           Household|     Medium| 42688.57|
|         Hard Drinks|      Small|  6487.94|
|               Dairy|       High| 12280.73|
|         Snack Foods|      Small| 48357.89|
|              Others|     Medium|  6647.48|
|              Others|      Small|  7583.91|
|         Snack Foods|     Medium| 60701.91|
|           Household|      Small| 39569.12|
|              Canned|     Medium| 29973.09|
|              Breads|       High|  3343.97|
|         

In [77]:
# Find the sum and average of the Item_MRP by each Item_Type and Outlet_Size
df.groupBy('Item_Type','Outlet_Size')\
    .agg(
      round(sum(col('Item_MRP')), 2).alias('Total_MRP'),
      round(avg(col('Item_MRP')), 2).alias('Average_MRP')
  )\
    .dropna(subset=['Outlet_Size'])\
    .show()

+--------------------+-----------+---------+-----------+
|           Item_Type|Outlet_Size|Total_MRP|Average_MRP|
+--------------------+-----------+---------+-----------+
|       Starchy Foods|     Medium|  7124.14|     148.42|
|Fruits and Vegeta...|     Medium| 59047.22|     142.97|
|        Frozen Foods|       High| 12588.29|     136.83|
|         Soft Drinks|       High|  6456.17|     131.76|
|           Breakfast|      Small|  3917.04|     130.57|
|                Meat|     Medium| 20326.45|     136.42|
|Fruits and Vegeta...|       High| 20671.35|     145.57|
|                Meat|       High|  5627.04|     137.24|
|        Baking Goods|       High|  9431.75|      129.2|
|           Household|     Medium| 42688.57|     147.71|
|         Hard Drinks|      Small|  6487.94|     129.76|
|               Dairy|       High| 12280.73|     153.51|
|         Snack Foods|      Small| 48357.89|     144.35|
|              Others|     Medium|  6647.48|     127.84|
|              Others|      Sma

## Collect_List()
Alternative to group_concat in SQL

In [78]:
data_cl = [
    ('user1','book1'),
    ('user1','book2'),
    ('user2','book2'),
    ('user2','book4'),
    ('user3','book1'),
]

schema_cl = StructType([
    StructField('user',StringType(),True),
    StructField('book',StringType(),True)
])

df_cl = spark.createDataFrame(data_cl,schema_cl)

df_cl.show()

+-----+-----+
| user| book|
+-----+-----+
|user1|book1|
|user1|book2|
|user2|book2|
|user2|book4|
|user3|book1|
+-----+-----+



In [80]:
df_cl.groupBy('user').agg(collect_list('book')).show()

+-----+------------------+
| user|collect_list(book)|
+-----+------------------+
|user1|    [book1, book2]|
|user2|    [book2, book4]|
|user3|           [book1]|
+-----+------------------+

